# Overview of Day 4
* Preliminaries
* What even is machine learning?
* Machine learning in Python: why scikit-learn?
* Feature extraction
* Feature selection
* Estimation
* Evaluation
* Automation


In [ ]:
# We'll load scikit-learn modules as we go,
# so we can see what we're using.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
%matplotlib inline

# Read in our preprocessed data set from Day 3.
# You may need to modify the path to the file
# depending on where you put it on your computer.
data = pd.read_csv('../data/preprocessed_data.csv')

# Since we'll be predicting outcomes, let's restrict
# to only common ones. It's hard to predict something
# we don't have very many training examples of.
data = data.groupby('outcome').filter(lambda x: len(x) >= 500)

# Let's also do some recoding to make life easier
data = data.dropna(subset=['age'])
categoricals = ['sex', 'sterilized']
data[categoricals] = data[categoricals].fillna('Unknown')

# Important, otherwise we have problems later
# when we try to concatenate based on index
data = data.reset_index()

# What even is machine learning?
<img src="https://s-media-cache-ak0.pinimg.com/736x/be/fc/cc/befcccae9891a505eabee71f7c808d4d.jpg" style="margin-bottom: 10px;">
* Better name might be "predictive modeling"
    * In contrast to traditional statistical approach, which might be called "explanatory modeling" [$^1$](https://projecteuclid.org/euclid.ss/1009213726),[$^2$](http://jakewestfall.org/publications/Yarkoni_Westfall_choosing_prediction.pdf)
    

* Traditional approach begins by positing a _data model_
    * e.g., linear regression model: $Y = \beta_0 + \beta_1X_1 + \ldots + \beta_pX_p + \epsilon$
    * ***"Assuming my model of how the data arose is correct, and given these parameter estimates, to what extent is variation in Y explained by variation in X?"***
    * Models should be interpretable in order to be useful
    

* In machine learning, overriding emphasis is just on whether we can accurately predict future Y values
    * ***"Regardless of what the true data model may be or what my parameter estimates look like, does my algorithm give outputs (Y) as close as possible to those of the true model when given the same inputs (X)?"***
    * If the model/algorithm is interpretable, that's a bonus, but not generally important
    * Premium placed instead on objective tests of accuracy in predicting new data

# Getting ahead of ourselves:
### Overfitting and in-sample vs. out-of-sample error
<img src="https://qph.ec.quoracdn.net/main-qimg-17ec84ff3f63f77f6b368f0eb6ef1890" style="margin-bottom: 10px;" width="1000px">

* How do we test for overfitting **before** the new data come in?
* By setting aside a fraction of our dataset -- no peeking! -- and pretending it's the future data
    * These set-aside data are called the *validation set*
* Example: We have 1000 samples in the full dataset. 
    * Fit the model to 900 samples (the training set)
    * Test how accurately the model predicts the remaining 100 samples (the validation set)
* This is the basic logic. In practice we often use a more sophisticated version of this called *cross-validation*. We'll return to this idea near the end of the class

# Machine learning in Python: why scikit-learn?
* There are hundreds of ML packages in Python
    * Theano, Tensorflow, orange, Pattern, PyMVPA, etc...
* But scikit-learn is dominant
    * Elegant, powerful interface
    * World-class documentation
    * Excellent performance
* The exception is deep learning--not supported in scikit-learn

# The typical predictive modeling pipeline
* Feature extraction/engineering
* Feature selection/dimensionality reduction
* Model/parameter selection
* Estimation
* Evaluation
* Rinse and repeat ad nauseam

# Feature extraction
* Deriving *new* (informative, non-redundant) predictors/features from the features you already have
* Or adding entirely new features to the dataset (e.g., web scraping for dog breed stats)
* Simple example of feature extraction: Adding a squared term

In [ ]:
# set up the plotting space
fig, axes = plt.subplots(1, 2, figsize=(15,6))

# estimate a simple linear regression model, plot its predictions
linear_model = sm.OLS(data['min_weight'], sm.add_constant(data['min_height']), missing='drop')
a, b = linear_model.fit().params
axes[0].scatter(data['min_height'], data['min_weight'], s=30)
axes[0].set(ylabel="Breed weight", xlabel="Breed height")
axes[0].set_title("Weight = b0 + b1*Height", size=20)
axes[0].plot([0, 30], [a, a + b*30], lw=3)

# now add Height-squared to the model and plot the model predictions
quad_data = sm.add_constant(pd.DataFrame({'h':data['min_height'], 'h2':data['min_height']**2}))
quadratic_model = sm.OLS(data['min_weight'], quad_data, missing='drop')
a, b, c = quadratic_model.fit().params
axes[1].scatter(data['min_height'], data['min_weight'], s=30)
axes[1].set(ylabel="Breed weight", xlabel="Breed height")
axes[1].set_title("Weight = b0 + b1*Height + b2*Height^2", size=20)
X = np.linspace(0, 30)
axes[1].plot(X, [a + b*x + c*x**2 for x in X], lw=3);

## Getting _all_ the features
* Well, probably not all... but a _lot_
* How much information can we get out of the original dataset?
* From an interpretation-oriented standpoint, maybe not much more
* From a machine learning standpoint, we've just scratched the surface
* Some things we could add: names, colors, any number of interactions...
    * The cost of trying out silly things is much lower
    * Multicollinearity is not (much of) a concern

## Bag-of-words approach to handling fur color
* If you have a lot of data, it's not always worth thinking deeply about your features
* E.g., how should we model fur color?
    * "Black/Tricolor", "Calico Point", "Brown Brindle/Blue Cream"
* Simple approach: treat color descriptions like a [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model)
* Extract all word tokens (possibly even N-grams)

In [ ]:
# How many colors in total?
print(data['color'].nunique())

# First 20 unique colors in the dataset
data['color'].unique()[:20]

### Applying the bag-of-words model

In [ ]:
# The CountVectorizer is an estimator that takes a series
# of documents (or strings) as input, and returns a count
# of every word token found in every document. There's also
# a TfidfVectorizer in cases where we want normalized frequency.
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer with default parameters.
# Some common arguments we might want to experiment with
# include min_df and max_df (which exclude words that are
# too frequent or infrequent), stop_words (which allows
# us to pass in a list of words to ignore), and ngram_range,
# which enables us to extract multi-word features.
vec = CountVectorizer()

# Extract all possible word features from the color list.
# Note that this returns a sparse matrix rather than a
# numpy array or a pandas DataFrame. A sparse matrix is
# a way of representing potentially very large 2-d arrays
# very efficiently, because we don't need to allocate
# memory for every cell in the array, only those that
# have a non-zero value.
fur_features = vec.fit_transform(data['color'])
print("fur_features is an object of type:", type(fur_features))

# After fitting, the names of the features (i.e., the
# columns of the sparse matrix returned by fit_transform())
# are stored in the estimator itself.
feature_names = vec.get_feature_names()

# Store in a pandas DF for easier manipulation later.
# Note that we convert the sparse array back to a dense
# one before loading into pandas. If our dataset were
# much bigger, we'd probably want to avoid this step
# and just keep working with the sparse matrix.
fur_features = pd.DataFrame(fur_features.todense(), columns=feature_names)

In [ ]:
# Let's take a look...
fur_features.head()
# We went from 534 unique color combinations to 37 binary color features

In [ ]:
# How many instances of each color?
fur_features.sum(axis=0).sort_values(ascending=False)

### Adding interaction terms
* We could dummy-code all our categorical variables and then take pairwise products
* But if we don't need interpretabilty, there's a simpler hack
    * Concatenate all variables for which we want interactions
    * Dummy-code the result
* Let's cross sex (2 categories), sterilization (2 categories), and breed (1787 categories)
    * Could result in up to $2 \times 2 \times 1787 = 7148$ new features! But since many of these combinations of categories likely contain 0 pets, it'll probably be fewer than that

In [ ]:
# Simply concatenate the columns we want--making sure to convert any numeric
# columns to string, otherwise the concatenation will fail.
data['ssb'] = data['sex'].astype(str) + '_' \
    + data['sterilized'].astype(str) + '_' + data['breed']

# How many unique levels?
num_levels = data['ssb'].nunique()
print("Total number of unique values: {}".format(num_levels))

# Now we can dummy-code the result
pd.get_dummies(data['ssb']).head(10)

### What else?
* Very easy to quickly build up thousands of derivative features in this way
* Doesn't mean we shouldn't think deeply about good features
    * Often, biggest jumps in performance are achieved by adding entirely new features (e.g., external dog breed data)
* Point is try to eke out every bit of signal from what we have

# Feature selection/reduction
* Not all features are created equal
* Just because we created 3,000+ features doesn't mean we need to include them all
* Two general approaches:
    1. Dimensionality reduction (extract latent signal from observed features)
        * Principal component analysis (PCA)
        * Independent component analysis (ICA)
        * etc.
    2. Feature selection (filter out features based on some criterion)
        * Keep high-variance features
        * Keep best-scoring features (i.e., strongest correlation with outcome)
        * Fit a preliminary estimator like lasso that drops some features
        * etc.
* Supported by the `decomposition` and `feature_selection` modules in `sklearn`

### Principal Component Analysis (PCA): The big idea
* For $P$ variables, we can plot the observations as a scatter plot in $P$-dimensional space
    * E.g., for $P=2$ features, we have a two-dimensional scatter plot with two axes, the X-axis and the Y-axis
* In PCA, we *rotate* the dataset in $P$-dimensional space to a *new* set of $P$ axes, called *principal components*, such that
    1. The observations are uncorrelated along the principal component axes
    2. The principal components are sorted in descending order of variance accounted for

In [ ]:
# Standardize the two features. It's usually 
# a good idea to do this prior to running PCA
df = data[['min_height', 'min_weight']].dropna()
df = (df - df.mean()) / df.std()

# compute the PCA using scikit-learn
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(df)

# set up the plotting space
fig, axes = plt.subplots(1, 2, figsize=(14,7))

# data points in original, unrotated features space
axes[0].scatter(df['min_height'], df['min_weight'], s=30)
axes[0].set_xlabel("Breed height")
axes[0].set_ylabel("Breed weight")
axes[0].set_ylim([-2, 3.5])
axes[0].set_xlim([-2, 3.5])
a, b = (0, pca.components_[0,1]/pca.components_[0,0])
axes[0].plot([-1.8, 2.4], [a + b*-1.8, a + b*2.4], lw=3)
a, b = (0, pca.components_[1,1]/pca.components_[1,0])
axes[0].plot([-.9, .9], [a + b*-.9, a + b*.9], lw=3)

# data points in rotated principal component (PC) space
rotated = pca.transform(df)
axes[1].scatter(-rotated[:,0], rotated[:,1], s=30)
axes[1].set_xlabel("Principal Component 1")
axes[1].set_ylabel("Principal Component 2")
axes[1].set_ylim([-2.5, 4])
axes[1].set_xlim([-2.5, 4])
axes[1].plot([-2.7, 3.5], [0, 0], lw=3)
axes[1].plot([0, 0], [-1.3, 1.3], lw=3);

# How much of the total variance is explained by each PC?
msg = "The first principal component alone explains {:.0%} of the total variance."
print(msg.format(pca.explained_variance_ratio_[0]))

* It's clear that the first principal component contains most of the useful information that was shared between Height and Weight
* So if we simply drop the second principal component, we've effectively reduced our two features down to one feature with minimal information loss

### Now let's do PCA on our array of over 3000 features
* PCA on such a large array would be very computationally demanding and take a long time
* So we'll do a faster, approximate version of PCA called *Randomized PCA*

In [ ]:
# First we need to recode our string column as a set of dummies
interaction_dummies = pd.get_dummies(data['ssb'])

# Let's concatenate this with the color features
# axis=1 indicates that we want to concatenate along
# the column axis (axis=0 would append each dataframe
# below the last.
lotsa_features = pd.concat([fur_features, interaction_dummies], axis=1)

# Like most other things in sklearn, decomposition classes
# implement the estimator interface. So they have fit() and
# predict() methods. Transformers also have a transform()
# method. First, we initialize the PCA transformer.
# We'll use a "randomized PCA" solver that is a speedier
# approximation of standard the principal component analysis
# (PCA) factorization. We need to specify the number of
# components we want at initialization. We'll take the first 100.
from sklearn.decomposition import PCA
rpca = PCA(100, svd_solver='randomized')

# Now we can fit and transform in one step
rpca_features = rpca.fit_transform(lotsa_features)

# How much of the variance do these components explain?

# set up the plotting space
fig, axes = plt.subplots(1, 2, figsize=(14,7))

# scree plot
axes[0].plot(rpca.explained_variance_ratio_, lw=3)
axes[0].set_title('Scree plot (% variance explained by each PC)', size=20)
axes[0].set_xlabel('Principal Component number')
axes[0].set_ylabel('% of total variance')

# cumulative variance explained
axes[1].plot(np.cumsum(rpca.explained_variance_ratio_), lw=3)
axes[1].set_title('Cumulative variance explained', size=20)
axes[1].set_xlabel('Principal Component number')
axes[1].set_ylabel('% of total variance');

# Estimation
* With features in hand, we can now fit some models!
* scikit-learn has a bewildering array of models
* We'll talk about model selection shortly
* First we'll look at one model/algorithm in detail: **K-nearest neighbors (KNN)**
    * Easy to understand
    * Often competitive with more complicated models
    * Example of an algorithm really only used in machine learning context (no *data model*)

## K-nearest neighbors (KNN) algorithm
<img src="http://en.proft.com.ua/media/science/r_knn_concept.png" style="margin-bottom: 10px;" width="1000px">

In [ ]:
from sklearn import neighbors
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from matplotlib.colors import ListedColormap

# some data pre-processing
df = data[['min_height','min_weight']]
df = (df - df.mean()) / df.std()                # standardize before applying PCA.
df['size'] = np.dot(df, pca.components_.T)[:,0] # add PC1 as 'size'.
df['outcome'] = data['outcome']                 # append outcome in string form
df['y'] = LabelEncoder().fit_transform(data['outcome']) # and integer form.
df['age'] = data['age']                         # add the age predictor.
df = df.dropna()                                # drop missing values.
df = df.loc[df['outcome'] != 'Died',]           # drop 'Died' outcome (too few cases, 0.4%)
for v in ['age','size']:                        # standardize the predictors. KNN falters
    df[v] = (df[v] - df[v].mean())/df[v].std()  # if predictors on wildly different scales.

# define function to compute KNN for a given K and plot the result
def knn_plot(k, ax):
    knn = neighbors.KNeighborsClassifier(k)     # load the KNN classifier
    knn.fit(df[['size','age']], df['y'])        # fit the KNN model
    predicted = knn.predict(df[['size','age']]) # retrieve the outcomes predicted by KNN
    acc = accuracy_score(df['y'], predicted)    # compute accuracy of those predictions
    
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    h = .1  # step size in the mesh
    x_min, x_max = df['size'].min() - 1, df['size'].max() + 1
    y_min, y_max = df['age'].min() - 1, df['age'].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])

    # Create color maps
    cmap_bold = ListedColormap(['red', 'green', 'blue', 'yellow'])
    cmap_light = ListedColormap(['pink', 'lightgreen', 'lightblue', 'lightyellow'])
    
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    ax.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    ax.scatter(df['size'], df['age'], c=df['y'], cmap=cmap_bold)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel('Breed size (standardized)')
    ax.set_ylabel('Dog age (standardized)')
    ax.set_title('K = {}; Accuracy = {:.1%}'.format(k, acc), size=20)
    
# set up the plotting space
fig, axes = plt.subplots(1, 3, figsize=(14,14/3))

# fit the models and make the plots
for k, ax in zip([2, 100, 2000], axes):
    knn_plot(k, ax)
axes[1].annotate(
    'Red = adoption, Green = euthanasia, Blue = transfer, Yellow = return to owner',
    xy=(0.5, 0), xytext=(0, -1),
    xycoords=('axes fraction', 'figure fraction'),
    textcoords='offset points',
    size=20, ha='center', va='bottom');

#### Is this good?
* Seems good, no?
* There are 4 classes, so chance should be 25%, right?
* Or should it...

In [ ]:
# Show the relative class frequencies
counts = df['outcome'].value_counts()
counts/counts.sum()

## We can (probably) do better
* Let's use some of that giant mess of features we extracted!
* Also, the analysis above only used the dogs (which are about half the dataset), so let's use the full dataset
* Finally, it's really easy to plug in other classifiers, so let's try some of those

In [ ]:
# For models with categoricals, we can call on
# patsy to dummy-code our variables, like we did
# when working with statsmodels
from patsy import dmatrix

# Some other estimators we can try (there are many more!)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC                      # support vector classifier
from sklearn.naive_bayes import BernoulliNB      # Naive Bayes for dummy predictors
from sklearn.linear_model import RidgeClassifier # ridge regression + classification rule

# plug in estimator of choice (just using default arguments)
# est = neighbors.KNeighborsClassifier() # default K = 5
est = RandomForestClassifier()
# est = DecisionTreeClassifier()
# est = SVC()
# est = BernoulliNB()
# est = RidgeClassifier()

# grab the outcome and encode it as integer rather than string
encoder = LabelEncoder()
y = encoder.fit_transform(data['outcome'])

# Set up the features, starting with 'age'
X = data['age']
X = np.c_[X.values, rpca_features[:,:20]]
# X = np.c_[X.values, lotsa_features]
for i in range(X.shape[1]): # standardize all predictors
    X[:,i] = (X[:,i] - X[:,i].mean())/X[:,i].std()

# fit the model/algorithm
est.fit(X, y)
y_predicted = est.predict(X)

# We're assessing accuracy *in the training set* so be skeptical!
accuracy_score(y, y_predicted)

# Evaluation
* Evaluating model performance is rarely straightforward
* There are many criteria we might value
* Simple answers can be misleading
* Let's take a look at _how_ we classified different outcomes

### Confusion matrix
* How does the classifier go wrong?

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y, y_predicted):
    ''' take true and predicted scores and plot confusion matrix '''
    # Get the confusion matrix
    cm = confusion_matrix(y, y_predicted)

    # Normalize the confusion matrix by dividing each row by its sum
    ncm = cm / cm.sum(axis=1)[:, np.newaxis]

    # Put into DataFrame and get all labels from the encoder
    class_labels = encoder.classes_
    ncm = pd.DataFrame(ncm, index=class_labels, columns=class_labels)

    # Rows are true classes, columns are assigned classes
    sns.heatmap(data=ncm, fmt='.2f', annot=True, cmap='Blues')

plt.figure(figsize=(8,7))
plot_confusion_matrix(y, y_predicted)
plt.gca().set_title('Confusion matrix\nRow = reality, Column = decision', size=20);

In [ ]:
# The classification report shows us performance for
# the most common metrics, by class
from sklearn.metrics import classification_report

print(classification_report(y, y_predicted))
print('Precision: Of all cases predicted to be X, how many really were X?')
print('Recall: Of all cases that really are X, how many were correctly predicted to be X?\n')

# which integer goes with which outcome?
# a cross-tab is a simple way to find out
print(pd.crosstab(data['outcome'], y))

## Cross-validation
* Hopefully performance now looks reasonable
* But there's still a potential problem: overfitting
* We're training and evaluating on the same dataset--this is a big no-no!
* scikit-learn provides easy ways to evaluate models out-of-sample
    * This is known as cross-validation

In [ ]:
# Import KFold cross-validation
from sklearn.cross_validation import KFold

# Placeholder for scores from each fold
scores = []

# Create k folds (in our case 5). Loop over the folds,
# and for each one, split the dataset into training and test.
# In each fold, we train the data on the training values,
# and then evaluate its performance on the test. Finally,
# we can take the average of the out-of-sample scores as
# our estimate of model performance.
folds = KFold(n=len(X), n_folds=5, shuffle=True)
print("Indices of training/test samples in each fold:")
for train, test in folds:
    print(train, test) # so we can see what's going on
    est.fit(X[train], y[train])
    pred_y = est.predict(X[test])
    fold_score = accuracy_score(y[test], pred_y)
    scores.append(fold_score)
    
# Overfitting be gone!
print("\nAccuracy in each fold:")
print(np.array(scores))

# Model selection
* We've experimented a lot with estimators, decomposition, evaluation, etc.
* This is only the tip of the iceberg...
* scikit-learn has hundreds of estimators!
* Two problems:
    1. How are we supposed to choose?
    2. How do we do this in a principled way?
* Basically we can try different things and see what leads to the highest cross-validation error
* Caveat: *ALL* of the steps in our analysis pipeline...
    * ...all data pre-processing strategies we try...
    * ...all models we try...
    * ...all hyperparameters (e.g., K parameter in KNN) we try for each model...
* ...should ideally occur within another cross-validation loop, an idea called *nested cross-validation*

# Fully automated pipelines
* We've done feature extraction, reduction, and selection; estimation; evaluation...
* But we can automate this
    * Both for efficiency, and to prevent overfitting (greatly facilitates nested cross-validation)
* sklearn.pipeline provides functionality for creating [fully automated Pipelines](http://scikit-learn.org/stable/modules/pipeline.html)
* We'll build a toy example with 2 steps, but we could chain our entire workflow

In [ ]:
# Import the Pipeline class
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest

# Initialize a list to store all the steps in our pipeline
steps = []

# Add feature selection
selector = SelectKBest(k=100)
steps.append(('select', selector))

# Add estimation
estimator = LogisticRegression()
steps.append(('estimate', estimator))

# set up the predictors
_X = np.c_[data['age'].values, lotsa_features]
for i in range(_X.shape[1]): # standardize all predictors
    _X[:,i] = (_X[:,i] - _X[:,i].mean())/_X[:,i].std()

# Initialize and fit the pipeline
pipeline = Pipeline(steps)
_y_pred = pipeline.fit(_X, y)

## Building our own estimator:
### The fit-predict interface
* scikit-learn is built around the estimator interface
* "An estimator is an object that fits a model based on some training data and is capable of inferring some properties on new data"
* Every estimator must implement fit() and predict() methods
* Makes it easy to extend scikit-learn with our own estimators

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class MercurialClassifier(BaseEstimator, ClassifierMixin):
    """Picks a random class and assigns that label to all cases."""

    def fit(self, X, y):
        ''' Selects a random class from the available options '''
        classes = np.unique(y)
        self.selected_ = np.random.choice(classes)
        return self

    def predict(self, X):
        ''' Applies the selected class to everything '''
        return np.repeat(self.selected_, len(X))

## A very quick dip into deep learning
* What is deep learning?
    * A field of machine learning that focuses on developing and applying "deep" neural network models
* Why deep learning?
    * The architecture of DNNs is (loosely) modeled on biological neural networks--which are very powerful!
    * For many real-world tasks (image recognition, language translation, etc.), deep learning blows everything else out of the water
    * A highly technical field
    * But also a lot of trial-and-error
    * Progress is *extremely* rapid

### Deep learning in Python
* Python is arguably the premier language for deep learning
* Virtually all major frameworks (TensorFlow, Caffe, Torch, etc.) have Python bindings
* High-level libraries like `keras` make things even easier
* We don't have time to do the topic justice, so let's just see a quick example

#### Logistic regression with neural networks
* Neural networks are extremely powerful function approximators
* We can start by training a logistic regression model using a neural net architecture
* Then we can start adding complexity (e.g., hidden layers)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Our y is currently a 1-D array of integers; for multiclass classification,
# keras's categorical_crossentropy loss expects a matrix of binary columns.
# We use keras's to_categorical utility (which does essentially the same)
# thing as pandas' get_dummies()) for the conversion.
y_binary = to_categorical(y)

# Set aside 20% of data as a hold-out test
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2)

n_classes = y_train.shape[1]

# # Multinomial logistic regression
model = Sequential([
    Dense(n_classes, activation='softmax', input_dim=X.shape[1])
])

# # Uncomment the next few lines for a deeper version of the model
# # that contains a hidden layer
# model = Sequential([
#     Dense(50, activation='sigmoid', input_dim=X.shape[1]),
#     Dense(n_classes, activation='softmax')
# ])

# Compile the model using a categorical cross-entry loss
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Shuffle the training data
perm = np.random.permutation(len(X_train))
X_train = X_train[perm]
y_train = y_train[perm]

# Print a summary of the model
model.summary()

# Fit the model!
model.fit(X_train, y_train, nb_epoch=100, validation_split=0.2)

### Going deeper...
* If we improved performance non-trivially just by adding one hidden layer, shouldn't we just keep adding more layers?
* In theory, perhaps, but in practice, probably not
* Why not?
    * Adding parameters makes the model harder to train, so we need more data
    * There's probably a fundamental limit to how predictable outcomes are given these data

### How did we do?
* We can evaluate the neural network's performance using the same tools we used before

In [ ]:
# Get predicted scores for the hold-out
y_pred = model.predict(X_test)

# The predictions are still in binary column format, so we need
# to squash them back into a vector of integers of class IDs.
# We can do this just by taking the max value in each row.
y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)

accuracy = (y_test_class == y_pred_class).mean()
print("Raw accuracy: {:.3f}\n".format(accuracy))

names = encoder.classes_
print("\nClassification report:\n")
print(classification_report(y_test_class, y_pred_class, target_names=names))

print("\nConfusion matrix:")
plot_confusion_matrix(y_test_class, y_pred_class)

# The End
* I hope you enjoyed the course and/or learned something useful!
* The links throughout the notebooks contain plenty of further resources
* Consider registering for [SciPy 2018](https://scipy2018.scipy.org) (here in July)--they have great [tutorials](https://scipy2018.scipy.org/ehome/299527/711308/)
* Feedback/suggestions for improvement is welcome
* Please fill out a course evaluation before you leave